# ADA MidSem P1

- Sampad Kumar Kar
- MCS202215

# 0. Imports

In [1]:
import os, sys
import pandas as pd
import numpy as np

# 1. Data Loading

In [2]:
data_path = os.path.join('data', 'raw', 'Dataset-Unicauca-Version2-87Atts.csv')

# read the data
df = pd.read_csv(data_path)

In [3]:
df.head()

,Flow.ID,Source.IP,Source.Port,Destination.IP,Destination.Port,Protocol,Timestamp,Flow.Duration,Total.Fwd.Packets,Total.Backward.Packets,...,Active.Std,Active.Max,Active.Min,Idle.Mean,Idle.Std,Idle.Max,Idle.Min,Label,L7Protocol,ProtocolName
0,172.19.1.46-10.200.7.7-52422-3128-6,172.19.1.46,52422,10.200.7.7,3128,6,26/04/201711:11:17,45523,22,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,131,HTTP_PROXY
1,172.19.1.46-10.200.7.7-52422-3128-6,10.200.7.7,3128,172.19.1.46,52422,6,26/04/201711:11:17,1,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,131,HTTP_PROXY
2,10.200.7.217-50.31.185.39-38848-80-6,50.31.185.39,80,10.200.7.217,38848,6,26/04/201711:11:17,1,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,7,HTTP
3,10.200.7.217-50.31.185.39-38848-80-6,50.31.185.39,80,10.200.7.217,38848,6,26/04/201711:11:17,217,1,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,7,HTTP
4,192.168.72.43-10.200.7.7-55961-3128-6,192.168.72.43,55961,10.200.7.7,3128,6,26/04/201711:11:17,78068,5,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,131,HTTP_PROXY


In [4]:
df.shape

(3577296, 87)

In [5]:
df.columns

Index(['Flow.ID', 'Source.IP', 'Source.Port', 'Destination.IP',
       'Destination.Port', 'Protocol', 'Timestamp', 'Flow.Duration',
       'Total.Fwd.Packets', 'Total.Backward.Packets',
       'Total.Length.of.Fwd.Packets', 'Total.Length.of.Bwd.Packets',
       'Fwd.Packet.Length.Max', 'Fwd.Packet.Length.Min',
       'Fwd.Packet.Length.Mean', 'Fwd.Packet.Length.Std',
       'Bwd.Packet.Length.Max', 'Bwd.Packet.Length.Min',
       'Bwd.Packet.Length.Mean', 'Bwd.Packet.Length.Std', 'Flow.Bytes.s',
       'Flow.Packets.s', 'Flow.IAT.Mean', 'Flow.IAT.Std', 'Flow.IAT.Max',
       'Flow.IAT.Min', 'Fwd.IAT.Total', 'Fwd.IAT.Mean', 'Fwd.IAT.Std',
       'Fwd.IAT.Max', 'Fwd.IAT.Min', 'Bwd.IAT.Total', 'Bwd.IAT.Mean',
       'Bwd.IAT.Std', 'Bwd.IAT.Max', 'Bwd.IAT.Min', 'Fwd.PSH.Flags',
       'Bwd.PSH.Flags', 'Fwd.URG.Flags', 'Bwd.URG.Flags', 'Fwd.Header.Length',
       'Bwd.Header.Length', 'Fwd.Packets.s', 'Bwd.Packets.s',
       'Min.Packet.Length', 'Max.Packet.Length', 'Packet.Length.Mean',
  

Out of these the following columns are of interest:

- `Flow.ID`:
This column likely represents a unique identifier for each network flow in the dataset. It is a nominal attribute that distinguishes different flows.
- `Source.IP` and `Destination.IP`:
`Source.IP` contains the IP address of the source of the network flow, while `Destination.IP` contains the IP address of the destination. These are nominal attributes representing network addresses.
'`Source.Port`' and `Destination.Port`:
'`Source.Port`' and `Destination.Port` typically represent the source and destination port numbers associated with the network communication. Port numbers are used to identify specific services or processes on a device.
- `Protocol`:
This column likely indicates the network protocol used for the communication in each flow. Common values include TCP, UDP, ICMP, etc. This is a categorical or nominal attribute.
- `Timestamp`:
This is a date-type attribute and represents the time at which the network flow occurred. It provides the temporal aspect of the flow data.
- `Flow.Duration`:
This represents the duration of the network flow in seconds. It measures how long the communication persisted between source and destination.
- `Flow.Bytes.s`:
This is the flow rate in bytes per second, representing the data transfer rate of the flow. This could help in understanding the rate of data transmission for a specific flow.

# 2. Data Summarization

## 2.1 Total no. of flows.

An unique flow is determined by an unique 6-tuple of the following attributes:

- `Source.IP`
- `Destination.IP`
- `Source.Port`
- `Destination.Port`
- `Protocol`
- `Timestamp`

In [6]:
columns_of_interest = ['Source.IP', 'Destination.IP', 'Source.Port', 'Destination.Port', 'Protocol', 'Timestamp']
unique_flows = df[columns_of_interest].drop_duplicates()

# extract the no. of unique flows
num_unique_flows = unique_flows.shape[0]
print("Total number of unique flows: ", num_unique_flows)

Total number of unique flows:  3141011


## 2.2 Total flow duration.

Total flow duration is just the sum of all the entries in the `Flow.Duration` column.

In [8]:
# total flow duration
total_flow_duration = df['Flow.Duration'].sum()
print("Total flow duration (in s): ", total_flow_duration)

Total flow duration (in s):  91015231179554


## 2.3 Total no. of bytes transferred.

Total bytes transferred per flow is the product of entries in `Flow.Bytes.s` and `Flow.Duration` respectively.

In [9]:
# add column named 'Flow.Transfer.Bytes' to the dataframe to store the total number of bytes transferred in each flow

df['Flow.Transfer.Bytes'] = df['Flow.Bytes.s'] * df['Flow.Duration']

# total number of bytes transferred
total_bytes_transferred = df['Flow.Transfer.Bytes'].sum()
print("Total number of bytes transferred: ", total_bytes_transferred)

Total number of bytes transferred:  4.696655193740449e+17


## 2.4 Big Flows.

We identify large flows in 3 ways:
- Big flows in terms of duration.
- Big flows in terms of bytes transferred.
- Big flows in terms of packets transferred.

### 2.4.1 Big flows in terms of duration.

In [11]:
# top 10 flows in terms of duration
top_10_flows_duration = df.sort_values(by='Flow.Duration', ascending=False).head(10)
top_10_flows_duration

,Flow.ID,Source.IP,Source.Port,Destination.IP,Destination.Port,Protocol,Timestamp,Flow.Duration,Total.Fwd.Packets,Total.Backward.Packets,...,Active.Max,Active.Min,Idle.Mean,Idle.Std,Idle.Max,Idle.Min,Label,L7Protocol,ProtocolName,Flow.Transfer.Bytes
3566617,10.200.7.196-52.202.201.151-37047-443-6,52.202.201.151,443,10.200.7.196,37047,6,15/05/201705:28:00,120000000,5,5,...,91911.0,142.0,3.996905e+07,1.819059e+07,59999443.0,24477987.0,BENIGN,178,AMAZON,6.570000e+08
2760107,192.168.220.5-10.200.7.5-1956-3128-6,10.200.7.5,3128,192.168.220.5,1956,6,11/05/201711:11:06,120000000,131,23,...,6572620.0,132182.0,2.143184e+07,9.477916e+06,28792537.0,5284109.0,BENIGN,131,HTTP_PROXY,1.555800e+10
2340852,192.168.60.56-10.200.7.6-59217-3128-6,10.200.7.6,3128,192.168.60.56,59217,6,11/05/201709:40:01,120000000,23,19,...,556411.0,49470.0,1.981018e+07,8.979766e+06,28120248.0,7837220.0,BENIGN,131,HTTP_PROXY,3.157000e+09
2564368,179.1.4.237-10.200.7.195-443-46591-6,10.200.7.195,46591,179.1.4.237,443,6,11/05/201710:39:39,120000000,33,31,...,2148607.0,109931.0,1.954283e+07,7.190379e+06,29850875.0,7884773.0,BENIGN,91,SSL,5.284000e+09
3248512,172.217.29.66-10.200.7.218-443-56678-6,10.200.7.218,56678,172.217.29.66,443,6,15/05/201711:15:22,120000000,45,59,...,853957.0,797500.0,5.917427e+07,3.401113e+04,59198321.0,59150222.0,BENIGN,126,GOOGLE,4.737100e+10
567829,192.168.29.6-10.200.7.7-62740-3128-6,192.168.29.6,62740,10.200.7.7,3128,6,27/04/201708:34:19,120000000,36,38,...,3000397.0,285.0,1.936969e+07,6.948755e+06,29537005.0,10189409.0,BENIGN,131,HTTP_PROXY,9.924000e+09
3048052,192.168.41.3-10.200.7.4-60406-3128-6,192.168.41.3,60406,10.200.7.4,3128,6,11/05/201703:41:31,120000000,13,15,...,143046.0,116211.0,3.987094e+07,1.173507e+04,39883301.0,39859952.0,BENIGN,131,HTTP_PROXY,6.076000e+09
2406736,192.173.28.37-10.200.7.194-80-51948-6,10.200.7.194,51948,192.173.28.37,80,6,11/05/201710:06:02,120000000,524,510,...,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,BENIGN,7,HTTP,7.688050e+11
981639,10.200.7.217-31.216.145.107-42426-80-6,31.216.145.107,80,10.200.7.217,42426,6,27/04/201710:38:51,120000000,6371,2947,...,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,BENIGN,7,HTTP,1.213340e+13
2931821,192.168.29.5-10.200.7.5-54332-3128-6,10.200.7.5,3128,192.168.29.5,54332,6,11/05/201703:27:50,120000000,27,18,...,4575052.0,3264.0,1.904108e+07,8.347807e+06,27453906.0,8969418.0,BENIGN,131,HTTP_PROXY,5.274000e+09


### 2.4.2 Big flows in terms of bytes transferred.

In [12]:
# top 10 flows in terms of bytes transferred
top_10_flows_bytes = df.sort_values(by='Flow.Transfer.Bytes', ascending=False).head(10)
top_10_flows_bytes

,Flow.ID,Source.IP,Source.Port,Destination.IP,Destination.Port,Protocol,Timestamp,Flow.Duration,Total.Fwd.Packets,Total.Backward.Packets,...,Active.Max,Active.Min,Idle.Mean,Idle.Std,Idle.Max,Idle.Min,Label,L7Protocol,ProtocolName,Flow.Transfer.Bytes
40680,192.168.180.51-10.200.7.4-57855-3128-6,192.168.180.51,57855,10.200.7.4,3128,6,26/04/201711:11:46,119267353,61894,542196,...,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,130,HTTP_CONNECT,1.346164e+15
504485,185.181.102.34-10.200.7.218-443-50731-6,10.200.7.218,50731,185.181.102.34,443,6,27/04/201708:26:40,119933704,113067,401387,...,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,91,SSL,8.767958e+14
367385,185.181.102.39-10.200.7.218-443-53313-6,10.200.7.218,53313,185.181.102.39,443,6,27/04/201707:56:25,119976864,102904,373823,...,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,91,SSL,8.438783e+14
688385,192.168.150.16-10.200.7.4-49908-3128-6,192.168.150.16,49908,10.200.7.4,3128,6,27/04/201709:10:55,95951303,50669,293916,...,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,130,HTTP_CONNECT,7.432624e+14
489249,185.181.102.39-10.200.7.217-443-45962-6,10.200.7.217,45962,185.181.102.39,443,6,27/04/201708:21:36,119999941,131664,329818,...,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,91,SSL,7.077779e+14
368971,185.181.102.39-10.200.7.218-443-58819-6,10.200.7.218,58819,185.181.102.39,443,6,27/04/201707:57:28,113805535,62007,315700,...,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,91,SSL,7.036114e+14
1766193,216.58.222.97-10.200.7.217-443-37798-6,10.200.7.217,37798,216.58.222.97,443,6,28/04/201710:09:35,101361539,215399,332120,...,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,126,GOOGLE,6.800715e+14
541440,185.181.102.40-10.200.7.218-443-59509-6,185.181.102.40,443,10.200.7.218,59509,6,27/04/201708:21:36,114114036,328695,119368,...,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,91,SSL,6.780236e+14
1890687,192.168.90.91-10.200.7.7-56726-3128-6,192.168.90.91,56726,10.200.7.7,3128,6,28/04/201710:09:35,101429751,184634,231893,...,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,126,GOOGLE,6.248774e+14
580761,192.168.72.31-10.200.7.8-56879-3128-6,192.168.72.31,56879,10.200.7.8,3128,6,27/04/201708:26:40,119999314,164711,170787,...,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,130,HTTP_CONNECT,5.979686e+14


### 2.4.3 Big flows in terms of packets transferred.

In [13]:
# add column named 'Total.Flow.Packets' to the dataframe to store the total number of packets transferred in each flow
df['Total.Flow.Packets'] = df['Flow.Packets.s'] * df['Flow.Duration']

# top 10 flows in terms of packets transferred
top_10_flows_packets = df.sort_values(by='Total.Flow.Packets', ascending=False).head(10)
top_10_flows_packets

,Flow.ID,Source.IP,Source.Port,Destination.IP,Destination.Port,Protocol,Timestamp,Flow.Duration,Total.Fwd.Packets,Total.Backward.Packets,...,Active.Min,Idle.Mean,Idle.Std,Idle.Max,Idle.Min,Label,L7Protocol,ProtocolName,Flow.Transfer.Bytes,Total.Flow.Packets
40680,192.168.180.51-10.200.7.4-57855-3128-6,192.168.180.51,57855,10.200.7.4,3128,6,26/04/201711:11:46,119267353,61894,542196,...,0.0,0.0,0.0,0.0,0.0,BENIGN,130,HTTP_CONNECT,1.346164e+15,6.040900e+11
1321186,192.168.142.22-10.200.7.9-50359-3128-6,10.200.7.9,3128,192.168.142.22,50359,6,27/04/201704:55:11,119735733,453190,115296,...,0.0,0.0,0.0,0.0,0.0,BENIGN,131,HTTP_PROXY,1.638852e+14,5.684860e+11
1766193,216.58.222.97-10.200.7.217-443-37798-6,10.200.7.217,37798,216.58.222.97,443,6,28/04/201710:09:35,101361539,215399,332120,...,0.0,0.0,0.0,0.0,0.0,BENIGN,126,GOOGLE,6.800715e+14,5.475190e+11
504485,185.181.102.34-10.200.7.218-443-50731-6,10.200.7.218,50731,185.181.102.34,443,6,27/04/201708:26:40,119933704,113067,401387,...,0.0,0.0,0.0,0.0,0.0,BENIGN,91,SSL,8.767958e+14,5.144540e+11
367385,185.181.102.39-10.200.7.218-443-53313-6,10.200.7.218,53313,185.181.102.39,443,6,27/04/201707:56:25,119976864,102904,373823,...,0.0,0.0,0.0,0.0,0.0,BENIGN,91,SSL,8.438783e+14,4.767270e+11
3319386,192.168.90.29-10.200.7.9-50081-3128-6,192.168.90.29,50081,10.200.7.9,3128,6,15/05/201711:11:01,119962375,141217,332673,...,38291080.0,12035642.0,0.0,12035642.0,12035642.0,BENIGN,126,GOOGLE,3.164889e+14,4.738900e+11
2387032,192.168.90.86-10.200.7.5-50478-3128-6,192.168.90.86,50478,10.200.7.5,3128,6,11/05/201709:40:28,119994179,115597,357008,...,0.0,0.0,0.0,0.0,0.0,BENIGN,126,GOOGLE,2.114648e+14,4.726050e+11
489249,185.181.102.39-10.200.7.217-443-45962-6,10.200.7.217,45962,185.181.102.39,443,6,27/04/201708:21:36,119999941,131664,329818,...,0.0,0.0,0.0,0.0,0.0,BENIGN,91,SSL,7.077779e+14,4.614820e+11
541440,185.181.102.40-10.200.7.218-443-59509-6,185.181.102.40,443,10.200.7.218,59509,6,27/04/201708:21:36,114114036,328695,119368,...,0.0,0.0,0.0,0.0,0.0,BENIGN,91,SSL,6.780236e+14,4.480630e+11
1890687,192.168.90.91-10.200.7.7-56726-3128-6,192.168.90.91,56726,10.200.7.7,3128,6,28/04/201710:09:35,101429751,184634,231893,...,0.0,0.0,0.0,0.0,0.0,BENIGN,126,GOOGLE,6.248774e+14,4.165270e+11
